In [10]:
from dotenv import load_dotenv
import os
import json
import asyncio
from agents import Agent, function_tool, Runner, trace, input_guardrail, output_guardrail, GuardrailFunctionOutput, RunContextWrapper
from datetime import datetime, timedelta
from openai import OpenAI
import requests
from typing import Dict, Union, List
import re


load_dotenv(override=True)

# Initialize OpenAI client - this was missing!
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [11]:
@function_tool
def explain_amadeus_response(amadeus_api_result: str, user_original_query: str = "") -> str:
    """
    Analyze and explain Amadeus API responses in beginner-friendly language.

    Args:
        amadeus_api_result: JSON string containing the full Amadeus API response
        user_original_query: The original user query for context (optional)

    Returns:
        Clear, beginner-friendly explanation of the flight data
    """

    print(f"\n📖 Explaining Amadeus API response...")
    if user_original_query:
        print(f"🎯 Original query context: '{user_original_query}'")

    # Enhanced system instructions for explaining flight data
    explainer_instructions = f"""You are a professional Amadeus GDS expert with many years of experience in the Flight Travel Industry.

Your task is to analyze the provided Amadeus API JSON response and create a clear, beginner-friendly explanation.

ANALYSIS REQUIREMENTS:
- Read and analyze the ENTIRE JSON response - do not skip any sections
- Focus on the most important information for travelers
- Explain technical terms in simple language
- Structure your response clearly with sections/bullet points

KEY AREAS TO EXPLAIN:
- Flight details (times, routes, airlines, flight numbers)
- Pricing information (total cost, currency, fare breakdown if available)
- Booking classes and cabin types
- Baggage allowances (if present)
- Any restrictions or conditions
- Duration and connections

EXPLANATION STYLE:
- Use simple, clear language
- Avoid technical jargon
- Organize information logically
- Highlight the most important details first
- If the response contains errors or is mock data, explain that clearly

CONTEXT: {user_original_query if user_original_query else "General flight search"}

Format your response with clear headings and bullet points for easy reading."""

    try:
        # Parse the JSON to validate it's proper JSON
        api_data = json.loads(amadeus_api_result)

        # Call OpenAI to explain the response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": explainer_instructions},
                {"role": "user", "content": f"Please explain this Amadeus API response:\n\n{amadeus_api_result}"}
            ],
            temperature=0.3
        )

        explanation = response.choices[0].message.content.strip()

        print("✅ Explanation generated successfully")
        return explanation

    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        return f"Error: The provided data is not valid JSON format. Please provide a proper Amadeus API response.\n\nError details: {str(e)}"
    except Exception as e:
        print(f"❌ Explanation error: {e}")
        return f"Error generating explanation: {str(e)}"

In [12]:
# Test the explain_amadeus_response function tool
# Example of how a main agent would use this tool

# Sample Amadeus API response (like what comes from the Query agent)
sample_amadeus_response = json.dumps({
    "status": "mock_success",
    "message": "Using mock data - credentials not available",
    "query_info": {
        "endpoint": "shopping/flight-offers",
        "parameters": {
            "originLocationCode": "ARN",
            "destinationLocationCode": "HAN",
            "departureDate": "2025-10-10",
            "adults": "1",
            "travelClass": "ECONOMY"
        }
    },
    "mock_flight_data": {
        "data": [{
            "type": "flight-offer",
            "id": "1",
            "itineraries": [{
                "segments": [{
                    "departure": {
                        "iataCode": "ARN",
                        "at": "2025-10-10T10:00:00"
                    },
                    "arrival": {
                        "iataCode": "HAN",
                        "at": "2025-10-10T18:30:00"
                    },
                    "carrierCode": "SK",
                    "number": "123"
                }]
            }],
            "price": {
                "currency": "EUR",
                "total": "299.00"
            }
        }],
        "meta": {"count": 1}
    }
})

# Test the function tool
original_query = "What are the booking classes for airline SK between ARN-HAN?"
explanation = explain_amadeus_response(sample_amadeus_response, original_query)
print(explanation)

TypeError: 'FunctionTool' object is not callable

In [16]:
# Create a simple test agent that will test the explainer function tool
# This agent will provide sample Amadeus API data and test the explanation quality

# More comprehensive test data - realistic Amadeus API response
comprehensive_test_data = json.dumps({
    "status": "success",
    "amadeus_response": {
        "meta": {
            "count": 2,
            "links": {
                "self": "https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=ARN&destinationLocationCode=HAN&departureDate=2025-10-10&adults=1&max=10"
            }
        },
        "data": [
            {
                "type": "flight-offer",
                "id": "1",
                "source": "GDS",
                "instantTicketingRequired": False,
                "nonHomogeneous": False,
                "oneWay": False,
                "lastTicketingDate": "2025-10-09",
                "numberOfBookableSeats": 7,
                "itineraries": [
                    {
                        "duration": "PT9H30M",
                        "segments": [
                            {
                                "departure": {
                                    "iataCode": "ARN",
                                    "terminal": "5",
                                    "at": "2025-10-10T10:00:00"
                                },
                                "arrival": {
                                    "iataCode": "CPH",
                                    "terminal": "3",
                                    "at": "2025-10-10T11:15:00"
                                },
                                "carrierCode": "SK",
                                "number": "1234",
                                "aircraft": {
                                    "code": "320"
                                },
                                "operating": {
                                    "carrierCode": "SK"
                                },
                                "duration": "PT1H15M",
                                "id": "1",
                                "numberOfStops": 0
                            },
                            {
                                "departure": {
                                    "iataCode": "CPH",
                                    "terminal": "3",
                                    "at": "2025-10-10T14:30:00"
                                },
                                "arrival": {
                                    "iataCode": "HAN",
                                    "terminal": "1",
                                    "at": "2025-10-11T07:30:00"
                                },
                                "carrierCode": "SK",
                                "number": "5678",
                                "aircraft": {
                                    "code": "359"
                                },
                                "operating": {
                                    "carrierCode": "SK"
                                },
                                "duration": "PT8H15M",
                                "id": "2",
                                "numberOfStops": 0
                            }
                        ]
                    }
                ],
                "price": {
                    "currency": "SEK",
                    "total": "8450.00",
                    "base": "7200.00",
                    "fees": [
                        {
                            "amount": "1250.00",
                            "type": "SUPPLIER"
                        }
                    ]
                },
                "pricingOptions": {
                    "fareType": [
                        "PUBLISHED"
                    ],
                    "includedCheckedBagsOnly": True
                },
                "validatingAirlineCodes": [
                    "SK"
                ],
                "travelerPricings": [
                    {
                        "travelerId": "1",
                        "fareOption": "STANDARD",
                        "travelerType": "ADULT",
                        "price": {
                            "currency": "SEK",
                            "total": "8450.00",
                            "base": "7200.00"
                        },
                        "fareDetailsBySegment": [
                            {
                                "segmentId": "1",
                                "cabin": "ECONOMY",
                                "fareBasis": "YOWSE",
                                "brandedFare": "ECONOMY_LIGHT",
                                "class": "Y",
                                "includedCheckedBags": {
                                    "quantity": 0
                                }
                            },
                            {
                                "segmentId": "2", 
                                "cabin": "ECONOMY",
                                "fareBasis": "YOWSE",
                                "brandedFare": "ECONOMY_LIGHT",
                                "class": "Y",
                                "includedCheckedBags": {
                                    "quantity": 0
                                }
                            }
                        ]
                    }
                ]
            },
            {
                "type": "flight-offer",
                "id": "2",
                "source": "GDS",
                "instantTicketingRequired": False,
                "nonHomogeneous": False,
                "oneWay": False,
                "lastTicketingDate": "2025-10-09",
                "numberOfBookableSeats": 4,
                "itineraries": [
                    {
                        "duration": "PT12H45M",
                        "segments": [
                            {
                                "departure": {
                                    "iataCode": "ARN",
                                    "terminal": "5",
                                    "at": "2025-10-10T16:45:00"
                                },
                                "arrival": {
                                    "iataCode": "FRA",
                                    "terminal": "1",
                                    "at": "2025-10-10T19:00:00"
                                },
                                "carrierCode": "LH",
                                "number": "9876",
                                "aircraft": {
                                    "code": "321"
                                },
                                "operating": {
                                    "carrierCode": "LH"
                                },
                                "duration": "PT2H15M",
                                "id": "3",
                                "numberOfStops": 0
                            },
                            {
                                "departure": {
                                    "iataCode": "FRA",
                                    "terminal": "1",
                                    "at": "2025-10-10T22:30:00"
                                },
                                "arrival": {
                                    "iataCode": "HAN",
                                    "terminal": "1",
                                    "at": "2025-10-11T13:30:00"
                                },
                                "carrierCode": "LH",
                                "number": "5432",
                                "aircraft": {
                                    "code": "359"
                                },
                                "operating": {
                                    "carrierCode": "LH"
                                },
                                "duration": "PT10H30M",
                                "id": "4",
                                "numberOfStops": 0
                            }
                        ]
                    }
                ],
                "price": {
                    "currency": "SEK",
                    "total": "9750.00",
                    "base": "8300.00",
                    "fees": [
                        {
                            "amount": "1450.00",
                            "type": "SUPPLIER"
                        }
                    ]
                },
                "pricingOptions": {
                    "fareType": [
                        "PUBLISHED"
                    ],
                    "includedCheckedBagsOnly": False
                },
                "validatingAirlineCodes": [
                    "LH"
                ],
                "travelerPricings": [
                    {
                        "travelerId": "1",
                        "fareOption": "STANDARD",
                        "travelerType": "ADULT",
                        "price": {
                            "currency": "SEK",
                            "total": "9750.00",
                            "base": "8300.00"
                        },
                        "fareDetailsBySegment": [
                            {
                                "segmentId": "3",
                                "cabin": "ECONOMY",
                                "fareBasis": "KPRSE",
                                "brandedFare": "ECONOMY_CLASSIC",
                                "class": "K",
                                "includedCheckedBags": {
                                    "quantity": 1,
                                    "weight": 23,
                                    "weightUnit": "KG"
                                }
                            },
                            {
                                "segmentId": "4",
                                "cabin": "ECONOMY", 
                                "fareBasis": "KPRSE",
                                "brandedFare": "ECONOMY_CLASSIC",
                                "class": "K",
                                "includedCheckedBags": {
                                    "quantity": 1,
                                    "weight": 23,
                                    "weightUnit": "KG"
                                }
                            }
                        ]
                    }
                ]
            }
        ]
    },
    "query_info": {
        "endpoint": "shopping/flight-offers",
        "parameters": {
            "originLocationCode": "ARN",
            "destinationLocationCode": "HAN",
            "departureDate": "2025-10-10",
            "adults": "1",
            "travelClass": "ECONOMY"
        }
    }
})

# Test Agent that uses the explainer function tool
test_agent_instructions = """
You are a Test Agent designed to use the explain_amadeus_response function tool.

Your tasks:
1. Use the explain_amadeus_response function to analyze the provided Amadeus API data
2. Use the explain_amadeus_response function to generate a complete detailed explenation of the given Amadeus API data to answer the inital user's question.
3. Provide a clear and final answer to the user's question, with great strucutre and simple language. 

Focus on:
- Clarity of the explanation
- Completeness of information covered
- Ease of understanding for beginners
- Organization and structure

User's Questions:
"What are the different booking classes for airline SK, ARN-HAN, use any date."
"""

test_agent = Agent(
    name="Explainer Test Agent",
    instructions=test_agent_instructions,
    tools=[explain_amadeus_response],
    model="gpt-4o-mini"
)

print("🧪 Starting test of the explainer function tool...")
print("📊 Test data includes: 2 flight offers, different airlines (SK & LH), pricing, baggage info")
print("=" * 80)

🧪 Starting test of the explainer function tool...
📊 Test data includes: 2 flight offers, different airlines (SK & LH), pricing, baggage info


In [17]:
# Run the test - let the test agent analyze the comprehensive test data
test_message = f"""
Please use the explain_amadeus_response function to analyze this Amadeus API response data. 

Test Data: {comprehensive_test_data}

Original User Query: "What are the different booking classes for airline SK, ARN-HAN, use any date."

After getting the explanation, please:
1. Make sure the explenation answers the original user query. 
2. Provide the answer back to the user in a clear and strucutred format.
3. Explain the Amadeus API answer in good detail, but beginner friendly. 
"""

# Run the test with trace for debugging
with trace("Explainer Test"):
    test_result = await Runner.run(test_agent, test_message)

print("\n" + "="*80)
print("🔍 TEST RESULTS:")
print("="*80)
print(test_result)
print("\n" + "="*80)


📖 Explaining Amadeus API response...
🎯 Original query context: 'What are the different booking classes for airline SK, ARN-HAN, use any date.'
✅ Explanation generated successfully

🔍 TEST RESULTS:
RunResult:
- Last agent: Agent(name="Explainer Test Agent", ...)
- Final output (str):
    ### Summary of Available Booking Classes for Airline SK (Scandinavian Airlines)
    Here's a clear overview of the flight options from **Stockholm (ARN)** to **Hanoi (HAN)** using the Amadeus API response for your query about booking classes.
    
    #### Overview
    There are **two flight options** available for your selected route. Here are the details for each flight.
    
    ---
    
    ### Flight Option 1 (Airline SK)
    - **Airline**: Scandinavian Airlines (SK)
    - **Flight Numbers**: 
      - **ARN to CPH**: SK 1234
      - **CPH to HAN**: SK 5678
    - **Departure**: 
      - From **ARN** (Stockholm) at **10:00 AM** on October 10, 2025
    - **Arrival**: 
      - At **HAN** (Hanoi) at **